### Implement From structured  Output

In [59]:
student_response = "pass"

In [60]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnableSequence
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [61]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser

In [62]:
class StuOutputCorrectness(BaseModel):
    sentiment: Literal['correct', 'partially_correct', 'incorrect'] = Field(description='Give the sentiment from the student response')
    
parser = PydanticOutputParser(pydantic_object=StuOutputCorrectness)

In [63]:
prompt = PromptTemplate(
    template="""
You are a tutor analyzing a student's attempt to solve a math problem.  The student's response is provided below.

Student Response: {response}

Based *solely* on the student's response, classify their answer as either: 'correct', 'partially_correct', or 'incorrect'.

{format_instruction}

*Important:* Provide *only* the JSON output specified in the format instructions. Do not include any other text.
""",
    input_variables=['response'],
    partial_variables={
        'format_instruction': parser.get_format_instructions()
    }
)

In [64]:
classifier_chain = prompt | model | parser

In [65]:
response = classifier_chain.invoke(
    {
        'response': "The sum of 2+2=5.0"
    }
)
response

StuOutputCorrectness(sentiment='incorrect')

In [66]:
response.sentiment

'incorrect'

In [67]:
classifier_chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
+----------------------+ 
| StuOutputCorrectness | 
+----------------------+ 


## *Build Conditional Branch*

In [96]:
prompt_correct = PromptTemplate(
    template="""
You are an AI tutor designing a prompt for a system to generate a response for a student. The student has correctly responded with: {response}. The original problem is: What is 2+2?.

Craft an adaptive prompt that will guide the system to:

1.  Briefly acknowledge the student's correct response and provide positive reinforcement (e.g., "Excellent!", "Correct!", "Great job!").
2.  Prompt the system to offer a concise question that leads the student to the next logical step in solving the original problem. The question should not give away the answer but encourage the student to apply their knowledge.

Example Adaptive Prompt:
"The student has correctly identified [previous step]. Now, prompt the student to explain how they would use this information to solve for [next unknown variable] in the original equation. Focus on eliciting the student's reasoning."
""",
    input_variables=['response']
)

prompt_partially_correct = PromptTemplate(
    template="""
You are an AI tutor designing a prompt for a system to generate a response for a student. The student's response, {response}, to the problem, what is 2+2?, is partially correct.

Craft an adaptive prompt that will guide the system to:

1.  Instruct the system to acknowledge that the student is on the right track but has made a specific error.
2.  Prompt the system to provide a hint or question that directly addresses the error, focusing on the underlying concept or calculation that was incorrect.
3.  Encourage the system to guide the student to revise their answer based on the hint.

Example Adaptive Prompt:
"The student has correctly applied [concept], but they made an error in [specific calculation]. Prompt the system to ask the student to double-check their work in that area and explain why [correct approach] is necessary. Guide the student to identify the correct result."
""",
    input_variables=['response']
)

prompt_incorrect = PromptTemplate(
    template="""
You are an AI tutor designing a prompt for a system to generate a response for a student. The student's response, {response}, to the problem, what is 2+2?, is incorrect.

Craft an adaptive prompt that will guide the system to:

1.  Instruct the system to avoid directly stating that the answer is wrong. Instead, prompt it to use language that gently suggests a need for review or a different approach.
2.  Prompt the system to offer a general hint or probing question that targets a fundamental concept related to the problem. This hint should help the student identify their misconception or gap in understanding.
3.  Guide the system to encourage the student to rethink the problem from a different perspective.

Example Adaptive Prompt:
"The student's response suggests a misunderstanding of [fundamental concept]. Prompt the system to ask the student to explain the principles of [fundamental concept] in their own words and how they relate to the given problem. Guide them to identify a more appropriate strategy for solving the problem."
""",
    input_variables=['response']
)

In [91]:
str_parser = StrOutputParser()

In [92]:
branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'correct', prompt_correct | model | str_parser),
    (lambda x: x.sentiment == 'partially_correct', prompt_partially_correct | model | str_parser),
    (lambda x: x.sentiment == 'incorrect', prompt_incorrect | model | str_parser),
    RunnableLambda(lambda x: "Could find any kind of sentiment.")
)

In [93]:
final_chain = classifier_chain | branch_chain

In [94]:
adaptive_prompt = final_chain.invoke(
    {
        'response': "the sum of 2+2=5",
    }
)

In [95]:
print(adaptive_prompt)

Adaptive Prompt:
"The student's response indicates a potential gap in understanding basic arithmetic. Prompt the system to guide the student by asking them to recount the process they used to arrive at their answer. Encourage the system to ask the student to consider the concept of addition as the combining of quantities and to reflect on what it means to add two numbers together. Guide the system to suggest the student to visualize the problem using objects or drawings to see the addition of 2 and 2 in a concrete way."


In [97]:
adaptive_prompt = final_chain.invoke(
    {
        'response': "the sum of 2+2=4",
    }
)

In [98]:
print(adaptive_prompt)

Great job! Now, let's think about how you can express the result of 2+2 in different ways, such as using objects or drawings. How might you do that?


In [74]:
final_chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     


## *****Access Gemma3:1B*****

In [75]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

In [76]:
llm = Ollama(model="gemma3:1b")  # Replace "llama3" with your model name

In [77]:
llm.invoke("hi")

"Hi there! How's your day going so far? 😊 \n\nIs there anything you’d like to talk about or need help with?"

In [78]:
chat_model = ChatOllama(model="gemma3:1b") # Replace "llama3" with your model name

In [79]:
chat_model.invoke("hi sir").content

'Hello there! How can I help you today? 😊'